In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from kaggle.api.kaggle_api_extended import KaggleApi

import warnings

In [ ]:
# Игнорирование предупреждений
warnings.filterwarnings('ignore')

In [ ]:
root_dir = os.getcwd()

kaggle_data_set = 'henrysue/online-shoppers-intention'

dataset_dir = 'dataset'
dataset_name = 'online_shoppers_intention.csv'
dataset_path = os.path.join(root_dir, dataset_dir, dataset_name)

separator = f'\n\b{'-'*60}\n\b'

In [ ]:
def load_data_csv(file_path):
    """
    Загрузка данных из CSV файла.
    :param file_path: Путь к CSV файлу.
    :return: DataFrame с загруженными данными.
    """
    try:
        return pd.read_csv(file_path)
    except Exception as e:
        raise Exception(e)

In [ ]:
def load_kaggle_data_set(dataset):
    """
    Загрузка датасета из kaggle.

    Важно: Необходимо предварительно получить API Token.
    Переместите файл kaggle.json в папку ~/.kaggle/ (для Linux и macOS)
    или в C:/Users/<Ваше_имя_пользователя>/.kaggle/ (для Windows).
    Если папка .kaggle не существует, создайте ее.
    :param dataset:
    :return:
    """

    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(dataset=dataset,
                               path=f'./{dataset_dir}',
                               force=True,
                               unzip=True)              # Загрузка и распаковка

In [ ]:
try:
    if not os.path.exists(dataset_path):
        # Загрузка файла датасета из kaggle если он не загружен
        load_kaggle_data_set(kaggle_data_set)
        # Загрузка файла в датафрейм
    df = load_data_csv(dataset_path)
except Exception as e:
    raise Exception (f'При выполнении произошла ошибка: {e}')

In [ ]:
# Экспорт датасета в Exel если не был создан ранее
dataset_exel = os.path.join(root_dir, 'dataset.xlsx')
if not os.path.isfile(dataset_exel):
    df.to_excel(dataset_exel, index=False)

In [ ]:
# Информация о датафрейме (датасете)
df.info()

In [ ]:
# Отсутствующие значения в столбцах датафрейма. Можно посмотреть и в информации, но так более наглядно.
# print(separator, 'Отсутствующие значения:\n', df.isna().sum())
df.isna().sum()

In [ ]:
# Вывод информации о дубликатах
print(separator, 'Количество дублирующихся значений: ', df.duplicated().sum())

In [ ]:
# Убрать дубликаты записей
df.drop_duplicates(inplace=True)

In [ ]:
# Базовые статистики
df.describe()

In [ ]:
df

In [ ]:
# Создание гистограмм для каждой числовой переменной
df.hist(figsize=(15, 15), color='skyblue', edgecolor='black')

# Словарь для названий
titles = {
    'Administrative'          : 'Административный',
    'Administrative_Duration' : 'Административный процесс',
    'Informational'           : 'Информационный',
    'Informational_Duration'  : 'Информационный процесс',
    'ProductRelated'          : 'Связанный с продуктом',
    'ProductRelated_Duration' : 'Связанный с продуктом процесс',
    'BounceRates'             : 'Количество отказов',
    'ExitRates'               : 'Количество просмотров',
    'PageValues'              : 'Значения страницы',
    'SpecialDay'              : 'Особый день',
    'Month'                   : 'Месяц',
    'OperatingSystems'        : 'Операционные системы',
    'Browser'                 : 'Браузер',
    'Region'                  : 'Регион',
    'TrafficType'             : 'Тип трафика',
    'VisitorType'             : 'Тип посетителя',
    'Weekend'                : 'Выходные',
    'Revenue'                 : 'Доход'
}
# Добавление названий для каждого графика и осей
for ax in plt.gcf().get_axes():
    ax.set_xlabel('Значение')
    ax.set_ylabel('Частота')
    old = ax.get_title()
    new = titles.get(old)
    if old:
        ax.set_title(ax.get_title().replace(old, new))

# Регулировка макета для предотвращения наложения подписей
plt.tight_layout()

In [ ]:
# Преобразование типов данных в числовые значения для возможности их обработки
df['Month'] = df['Month'].map({ 'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4,
                                    'May': 5, 'June': 6,'Jul': 7, 'Aug': 8,
                                    'Sep': 9, 'Oct': 10,'Nov': 11,'Dec': 12})

In [ ]:

df['VisitorType'] = df['VisitorType'].astype('category')
df['Weekend'] = df['Weekend'].astype('category')
df['VisitorType'] = df['VisitorType'].cat.codes
df['Weekend'] = df['Weekend'].astype('int')
df['Revenue'] = df['Revenue'].astype('int')


In [ ]:
# Преобразованный дата фрейм
df

In [ ]:
# Посмотрим оказывает ли месяц влияние на продажи и если да, то какое
colors = sns.color_palette('viridis', len(df['Month'].unique()))
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='Month', hue='Month', palette=colors)
plt.title('График количества продаж по месяцам', fontsize=16)
plt.xlabel('Месяц', fontsize=14)
plt.ylabel('Количество продаж', fontsize=14)
plt.show()

In [ ]:
# Наиболее активные продажи в мае, ноябре, марте и декабре.

In [ ]:
# Посмотрим матрицу корреляции
corr = df.corr()
mask = np.triu(np.ones_like(corr), k=0)

plt.figure(figsize=(20, 14))
sns.heatmap(corr, mask=mask, annot=True, cmap='coolwarm')
plt.title('Тепловая карта матрицы корреляции')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Разделение на признаки и целевую переменную
X = df.drop("Revenue", axis=1)
y = df["Revenue"]

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Список итоговых результатов
results = []

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора градиентного бустинга
# ---------------------------------------------------------

# Импорт классификатора градиентного бустинга
from sklearn.ensemble import GradientBoostingClassifier

# Создаю модель
model_gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Тренирую модель
model_gbc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_gbc.predict(X_test)

# Отчет классификации
report_gbc = classification_report(y_test, y_pred)
report_gbc_d = classification_report(y_test, y_pred, output_dict=True)

# Матрица ошибок
conf_matrix_gbc = confusion_matrix(y_test, y_pred)

accuracy_gbc= accuracy_score(y_test, y_pred)
acc_d = accuracy_score(y_test, y_pred)
results.append()

In [ ]:
print(separator,'\bДля GradientBoostingClassifier:', separator)
print('Матрица ошибок\n',conf_matrix_gbc)
print(f'\nAccuracy: {accuracy_gbc:.2f}')
print('\nКлассификационный отчет\n',report_gbc, separator)
report_gbc_d

In [ ]:
# Определение важности признаков
feature_importances = model_gbc.feature_importances_
features = X.columns

In [ ]:
# Визуализация важности признаков градиентного бустинга
plt.figure(figsize=(10, 6))
plt.xticks(rotation=45, ha='right')
plt.bar(features, feature_importances)
plt.ylabel('Важность признака')
plt.xlabel('Название признака')
plt.title('Важность признаков в модели градиентного бустинга')
plt.show()

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора категориального бустинга (Categorical Boosting)
# ---------------------------------------------------------


# Импорт классификатора CatBoost
from catboost import CatBoostClassifier

# Создаю модель
model_cbc= CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, random_state=42, verbose=False)

# Тренирую модель
model_cbc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_cbc.predict(X_test)

# Отчет классификации
report_cbc = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_cbc = confusion_matrix(y_test, y_pred)

accuracy_cbc= accuracy_score(y_test, y_pred)

In [ ]:
print(separator, '\bДля категориального бустинга (Categorical Boosting):', separator)
print('Матрица ошибок\n', conf_matrix_cbc)
print(f'\nAccuracy: {accuracy_cbc:.2f}')
print('\nКлассификационный отчет\n', report_cbc, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора адаптивного бустинга AdaBoostClassifier
# ---------------------------------------------------------

# Импорт классификатора AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier


# Создаю модель
model_abc = AdaBoostClassifier(
    learning_rate=0.1,
    random_state=42
)

# Тренирую модель
model_abc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_abc.predict(X_test)

# Отчет классификации
report_abc = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_abc = confusion_matrix(y_test, y_pred)

accuracy_abc= accuracy_score(y_test, y_pred)

In [ ]:
print(separator, '\bДля адаптивного бустинга (AdaBoostClassifier):', separator)
print('Матрица ошибок\n', conf_matrix_abc)
print(f'\nAccuracy: {accuracy_abc:.2f}')
print('\nКлассификационный отчет\n', report_abc, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора ExtraTreesClassifier
# ---------------------------------------------------------

# Импорт классификатора ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier


# Создаю модель
model_etc = ExtraTreesClassifier(n_estimators=100, max_features='sqrt', random_state=42)

# Тренирую модель
model_etc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_etc.predict(X_test)

# Отчет классификации
report_etc = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_etc = confusion_matrix(y_test, y_pred)

accuracy_etc= accuracy_score(y_test, y_pred)


In [ ]:
print(separator, '\bДля  (ExtraTreesClassifier):', separator)
print('Матрица ошибок\n', conf_matrix_etc)
print(f'\nAccuracy: {accuracy_etc:.2f}')
print('\nКлассификационный отчет\n', report_etc, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора квадратичного дискриминантного анализа QuadraticDiscriminantAnalysis
# ---------------------------------------------------------

# Импорт классификатора QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Создаю модель
model_qda = QuadraticDiscriminantAnalysis()

# Тренирую модель
model_qda.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_qda.predict(X_test)

# Отчет классификации
report_qda = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_qda = confusion_matrix(y_test, y_pred)

accuracy_qda = accuracy_score(y_test, y_pred)


In [ ]:
print(separator, '\bДля  квадратичного дискриминантного анализа  (QuadraticDiscriminantAnalysis):', separator)
print('Матрица ошибок\n', conf_matrix_qda)
print(f'\nAccuracy: {accuracy_qda:.2f}')
print('\nКлассификационный отчет\n', report_qda, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора Light Gradient Boosting Machine
# ---------------------------------------------------------

# Импорт классификатора lightgbm
import lightgbm as lgb

# Создаю модель
model_lgbmc = lgb.LGBMClassifier(num_leaves=31, learning_rate=0.05, n_estimators=100)

# Тренирую модель
model_lgbmc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_lgbmc.predict(X_test)

# Отчет классификации
report_lgbmc = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_lgbmc = confusion_matrix(y_test, y_pred)

accuracy_lgbmc = accuracy_score(y_test, y_pred)


In [ ]:
print(separator, '\bДля (Light Gradient Boosting Machine):', separator)
print('Матрица ошибок\n', conf_matrix_qda)
print(f'\nAccuracy: {accuracy_qda:.2f}')
print('\nКлассификационный отчет\n', report_qda, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора KNeighborsClassifier
# ---------------------------------------------------------

# Импорт классификатора KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Создаю модель
model_knc = KNeighborsClassifier(n_neighbors=6)  # Задаем количество соседей. При k=6 удалось повысить Accuracy до 0.84

# Тренирую модель
model_knc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_knc.predict(X_test)

# Отчет классификации
report_knc = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_knc = confusion_matrix(y_test, y_pred)

accuracy_knc = accuracy_score(y_test, y_pred)


In [ ]:
print(separator, '\bДля K-ближайших (KNeighborsClassifier):', separator)
print('Матрица ошибок\n', conf_matrix_knc)
print(f'\nAccuracy: {accuracy_knc:.2f}')
print('\nКлассификационный отчет\n', report_knc, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора DecisionTreeClassifier
# ---------------------------------------------------------

# Импорт классификатора DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Попробую взять только несколько признаков.
# При выборе указанных признаков получил максимум Accuracy = 0.88
sel_cols = [
              # 'Administrative',
              'Administrative_Duration',
              # 'Informational',
              # 'Informational_Duration',
              # 'ProductRelated',
              # 'ProductRelated_Duration',
              # 'BounceRates',
              # 'ExitRates',
              'PageValues',
              # 'SpecialDay',
              'Month',
              # 'OperatingSystems',
              # 'Browser',
              # 'Region',
              # 'TrafficType',
              'VisitorType',
              'Weekend',
              ]
X_dtc_2 = df[sel_cols]
y_dtc_2 = df.Revenue

# Разделение данных на обучающий и тестовый наборы
X_train_dtc_2, X_test_dtc_2, y_train_dtc_2, y_test_dtc_2 = train_test_split(X_dtc_2, y_dtc_2, test_size=0.2, random_state=42)

# Создаю модель
model_dtc = DecisionTreeClassifier(random_state=42)
model_dtc_2 = DecisionTreeClassifier(random_state=42)

# Тренирую модель
model_dtc.fit(X_train, y_train)
model_dtc_2.fit(X_train_dtc_2, y_train_dtc_2)

# Предсказание на тестовой выборке
y_pred = model_dtc.predict(X_test)
y_pred_dtc_2 = model_dtc_2.predict(X_test_dtc_2)

# Отчет классификации
report_dtc = classification_report(y_test, y_pred)
report_dtc_2 = classification_report(y_test_dtc_2, y_pred_dtc_2)

# Матрица ошибок
conf_matrix_dtc = confusion_matrix(y_test, y_pred)
conf_matrix_dtc_2 = confusion_matrix(y_test_dtc_2, y_pred_dtc_2)

accuracy_dtc = accuracy_score(y_test, y_pred)
accuracy_dtc_2 = accuracy_score(y_test_dtc_2, y_pred_dtc_2)


In [ ]:
print(separator, '\bДля (DecisionTreeClassifier):', separator)
print('Матрица ошибок при выборе всех полей\n', conf_matrix_dtc)
print(f'Матрица ошибок при выборе полей {sel_cols}\n', conf_matrix_dtc_2)
print(f'\nAccuracy при выборе всех полей: {accuracy_dtc:.2f}')
print(f'\nAccuracy: {accuracy_dtc_2:.2f}')
print('\nКлассификационный отчет при выборе всех полей\n', report_dtc, separator)
print(f'\nКлассификационный отчет\n', report_dtc_2, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора экстремального градиентного бустинга (XGBClassifier)
# ---------------------------------------------------------

# Импорт классификатора XGBClassifier
import xgboost as xgb

# Создаю модель
model_xgbc = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Тренирую модель
model_xgbc.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_xgbc.predict(X_test)

# Отчет классификации
report_xgbc = classification_report(y_test, y_pred)

# Матрица ошибок
conf_matrix_xgbc = confusion_matrix(y_test, y_pred)

accuracy_xgbc= accuracy_score(y_test, y_pred)


In [ ]:
print(separator, '\bДля классификатора экстремального градиентного бустинга (XGBClassifier):', separator)
print('Матрица ошибок\n', conf_matrix_xgbc)
print(f'\nAccuracy: {accuracy_xgbc:.2f}')
print('\nКлассификационный отчет\n', report_xgbc, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация фиктивного классификатора (DummyClassifier)
# с параметрами uniform, stratified
# ---------------------------------------------------------

# Импорт классификатора DummyClassifier
from sklearn.dummy import DummyClassifier

# Создаю модель
model_dc_uniform = DummyClassifier(strategy="uniform")
model_dc_stratified = DummyClassifier(strategy="stratified")

# Тренирую модель
model_dc_uniform.fit(X_train, y_train)
model_dc_stratified.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_uniform = model_dc_uniform.predict(X_test)
y_pred_stratified = model_dc_stratified.predict(X_test)

# Отчет классификации
report_dc_uniform = classification_report(y_test, y_pred_uniform)
report_dc_stratified = classification_report(y_test, y_pred_stratified)

# Матрица ошибок
conf_matrix_dc_uniform = confusion_matrix(y_test, y_pred_uniform)
conf_matrix_dc_stratified = confusion_matrix(y_test, y_pred_stratified)

accuracy_dc_uniform = accuracy_score(y_test, y_pred_uniform)
accuracy_dc_stratified = accuracy_score(y_test, y_pred_stratified)


In [ ]:
print(separator, '\bДля фиктивного классификатора (DummyClassifier):', separator)
print('Матрица ошибок с параметром uniform\n', conf_matrix_dc_uniform)
print('Матрица ошибок с параметром stratified\n', conf_matrix_dc_stratified)
print(f'\nAccuracy с параметром uniform: {accuracy_dc_uniform:.2f}')
print(f'\nAccuracy с параметром stratified: {accuracy_dc_stratified:.2f}')
print('\nКлассификационный отчет с параметром uniform\n', report_dc_uniform)
print('\nКлассификационный отчет с параметром uniform\n', report_dc_stratified, separator)

In [ ]:
# ---------------------------------------------------------
# Реализация классификатора SVM - линейное ядро.
# ---------------------------------------------------------

# Импорт классификатора SVC
from sklearn.svm import SVC

# Создаю модели
model_svc_poly =  SVC(kernel='poly', random_state=42) # Полиномиальное ядро (Polynomial Kernel) создает границы разделения, которые могут быть более сложными, чем линейные, но менее гибкими, чем у RBF
model_svc_rbf =  SVC(kernel='rbf', random_state=42) # Ядро RBF (Radial Basis Function) создает гладкие, не линейные границы разделения.

# Тренирую модели
model_svc_poly.fit(X_train, y_train)
model_svc_rbf.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_svc_poly = model_svc_poly.predict(X_test)
y_pred_svc_rbf = model_svc_rbf.predict(X_test)

# Отчет классификации
report_svc_poly = classification_report(y_test, y_pred_svc_poly)
report_svc_rbf = classification_report(y_test, y_pred_svc_rbf)

# Матрица ошибок
conf_matrix_svc_poly = confusion_matrix(y_test, y_pred_svc_poly)
conf_matrix_svc_rbf = confusion_matrix(y_test, y_pred_svc_rbf)

accuracy_svc_poly = accuracy_score(y_test, y_pred_svc_poly)
accuracy_svc_rbf = accuracy_score(y_test, y_pred_svc_rbf)


In [ ]:
print(separator, '\bДля классификатора SVM - линейное ядро (SVC):', separator)
print('Матрица ошибок с параметром kernel="poly"\n', conf_matrix_svc_poly)
print('Матрица ошибок с параметром kernel="rbf"\n', conf_matrix_svc_rbf)
print(f'\nAccuracy с параметром kernel="poly": {accuracy_svc_poly:.2f}')
print(f'\nAccuracy с параметром kernel="rbf": {accuracy_svc_rbf:.2f}')
print('\nКлассификационный отчет с параметром kernel="poly"\n', report_svc_poly)
print('\nКлассификационный отчет с параметром kernel="rbf"\n', report_svc_rbf, separator)

In [ ]:
# Вывод: оба ядра показали практически одинаковые результаты

In [ ]:
# Итоговая визуализация
plt.figure(figsize=(12, 6))
sns.barplot(data=results_df.melt(id_vars="Model", value_vars=["F1 Score", "Accuracy", "ROC AUC"]),
            x="value", y="Model", hue="variable")
plt.title("Сравнение метрик классификаторов")
plt.xlabel("Значение метрики")
plt.ylabel("Модель")
plt.legend(title="Метрика")
plt.tight_layout()
plt.show()

In [ ]:
# Секция 1: Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Секция 2: Загрузка данных
categorical_cols = df.select_dtypes(include="object").columns
df[categorical_cols] = df[categorical_cols].apply(LabelEncoder().fit_transform)

X = df.drop("Revenue", axis=1)
y = df["Revenue"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Секция 3: Классификаторы
models = {
    "Gradient Boosting": GradientBoostingClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "LightGBM": LGBMClassifier(),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "Dummy": DummyClassifier(),
    "Linear SVM": SVC(kernel='linear', probability=True),
}

# Секция 4: Обучение и сравнение
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else np.zeros_like(y_pred)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "ROC AUC": roc_auc_score(y_test, y_proba),
    })

results_df = pd.DataFrame(results).sort_values(by="F1 Score", ascending=False).reset_index(drop=True)

# Секция 5: Вывод результатов
print("=== Сравнение моделей ===")
print(results_df)

# Секция 6: Визуализация
plt.figure(figsize=(12, 6))
sns.barplot(data=results_df.melt(id_vars="Model", value_vars=["F1 Score", "Accuracy", "ROC AUC"]),
            x="value", y="Model", hue="variable")
plt.title("Сравнение метрик классификаторов")
plt.xlabel("Значение метрики")
plt.ylabel("Модель")
plt.legend(title="Метрика")
plt.tight_layout()
plt.show()
